In [1]:
import pandas as pd

df = pd.read_csv("clean_EventsAdapt_human_dat.csv")
# exclude AAR control sentences
df = df[df["TrialType"] != "AAR"]
df

,WorkerId,Score,Plausibility,ItemNum,Sentence,TrialType,Voice
0,A12FTSX85NQ8N9,4.0,Plausible,1,The raider caught the illness.,AI,active
1,A18XFOKU5G1OL1,4.0,Plausible,1,The raider caught the illness.,AI,active
2,A1M24593XO5Z3C,4.0,Plausible,1,The illness was caught by the raider.,AI,passive
3,A1R780RJYQ68GY,7.0,Plausible,1,The illness was caught by the raider.,AI,passive
4,A1TEXCUTI4IUUN,7.0,Plausible,1,The illness was caught by the raider.,AI,passive
...,...,...,...,...,...,...,...
31537,A3RLCGRXA34GC0,2.0,Implausible,368,The warmonger was terrorized by the gunsmith.,AAN,passive
31538,A98S1WLZ7NJL0,3.0,Implausible,368,The warmonger was terrorized by the gunsmith.,AAN,passive
31539,AFDC9A6Z60W2Z,7.0,Implausible,368,The warmonger was terrorized by the gunsmith.,AAN,passive
31540,ALEE1QD4TW9G4,7.0,Implausible,368,The warmonger was terrorized by the gunsmith.,AAN,passive


In [2]:
# get avg score
df = df.groupby(["ItemNum", "Voice", "TrialType", "Plausibility", "Sentence"])["Score"].mean().reset_index()
# add minimal pair column
df["MinimalPairNr"] = [num for num in range(1, int(len(df) / 2) + 1) for _ in range(2)]
df

,ItemNum,Voice,TrialType,Plausibility,Sentence,Score,MinimalPairNr
0,1,active,AI,Implausible,The illness caught the raider.,3.083333,1
1,1,active,AI,Plausible,The raider caught the illness.,5.545455,1
2,1,passive,AI,Implausible,The raider was caught by the illness.,2.541667,2
3,1,passive,AI,Plausible,The illness was caught by the raider.,5.333333,2
4,2,active,AI,Implausible,The disease contracted the marauder.,1.440000,3
...,...,...,...,...,...,...,...
1023,367,passive,AAN,Plausible,The lab assistant was overwhelmed by the neuro...,6.782609,512
1024,368,active,AAN,Implausible,The gunsmith terrorized the warmonger.,4.409091,513
1025,368,active,AAN,Plausible,The warmonger terrorized the gunsmith.,5.120000,513
1026,368,passive,AAN,Implausible,The warmonger was terrorized by the gunsmith.,4.700000,514


In [3]:
data = []
for item_num in sorted(df.MinimalPairNr.unique()):
    rows = df[df.MinimalPairNr==item_num]
    good = rows[rows.Plausibility=="Plausible"].squeeze()
    bad = rows[rows.Plausibility=="Implausible"].squeeze()

    good_sentence = good.Sentence
    bad_sentence = bad.Sentence
    
    data.append(dict(
        item_id=item_num,
        good_sentence=good_sentence,
        bad_sentence=bad_sentence,
        good_human_score=good.Score,
        bad_human_score=bad.Score,
        trial_type=good.TrialType,
        voice=good.Voice
    ))

clean_df = pd.DataFrame(data)
clean_df.to_csv("corpus.csv", index=False)

In [4]:
clean_df

,item_id,good_sentence,bad_sentence,good_human_score,bad_human_score,trial_type,voice
0,1,The raider caught the illness.,The illness caught the raider.,5.545455,3.083333,AI,active
1,2,The illness was caught by the raider.,The raider was caught by the illness.,5.333333,2.541667,AI,passive
2,3,The marauder contracted the disease.,The disease contracted the marauder.,5.680000,1.440000,AI,active
3,4,The disease was contracted by the marauder.,The marauder was contracted by the disease.,5.875000,1.818182,AI,passive
4,5,The convict dreaded the prison.,The prison dreaded the convict.,6.208333,4.318182,AI,active
...,...,...,...,...,...,...,...
509,510,The abused woman was comforted by the social w...,The social worker was comforted by the abused ...,6.681818,3.320000,AAN,passive
510,511,The neuroscientist overwhelmed the lab assistant.,The lab assistant overwhelmed the neuroscientist.,5.833333,5.636364,AAN,active
511,512,The lab assistant was overwhelmed by the neuro...,The neuroscientist was overwhelmed by the lab ...,6.782609,5.375000,AAN,passive
512,513,The warmonger terrorized the gunsmith.,The gunsmith terrorized the warmonger.,5.120000,4.409091,AAN,active
